In [1]:
from utils import all_features, all_non_weather_features
from tqdm import tqdm
import pickle
import os
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
df = pd.read_parquet("../data/silver/water_consumption_silver.parquet")
df[df.inputed_row==False].head()

,timestamp,year,month,day,day_of_week,week_of_year,hour,input_flow_rate_first,input_flow_rate_last,input_flow_rate_mean,reservoir_level_percentage_first,reservoir_level_percentage_last,reservoir_level_percentage_mean,output_flow_rate_first,output_flow_rate_last,output_flow_rate_mean,pressure_first,pressure_last,pressure_mean,pump_1_duration_sum,pump_2_duration_sum,air_temp_c,total_precip_mm,relative_humidity_percentage,input_flow_rate_diff,output_flow_rate_diff,reservoir_level_change,pressure_change,pump_1_active,pump_2_active,change_from_last_hour_output_flow_rate_mean,inputed_row
1,2023-03-17 13:00:00,2023,3,17,4,11,13,65.64,65.24,65.587143,39.00,44.40,41.617143,45.640000,49.240000,50.704846,38.13,38.27,38.181429,0.000000,3600.00000,28.9,0.0,59.0,-0.40,3.600000,5.40,0.14,False,True,1.014115,False
2,2023-03-17 14:00:00,2023,3,17,4,11,14,66.46,65.64,65.654000,44.40,49.86,46.736500,66.460000,14.858750,50.261628,38.40,38.63,38.459000,0.000000,3600.00000,29.9,0.0,54.0,-0.82,-51.601250,5.46,0.23,False,True,-0.443218,False
3,2023-03-17 15:00:00,2023,3,17,4,11,15,65.64,33.74,49.395000,50.56,53.80,52.405000,42.306667,93.740000,73.561667,38.70,38.40,38.515000,145.788337,3600.00000,30.4,0.0,53.0,-31.90,51.433333,3.24,-0.30,True,True,23.300038,False
4,2023-03-17 16:00:00,2023,3,17,4,11,16,37.92,0.00,3.792000,53.80,38.76,45.368000,37.920000,56.000000,46.141631,38.50,31.12,32.054500,0.000000,667.39255,31.2,0.0,57.0,-37.92,18.080000,-15.04,-7.38,False,True,-27.420036,False
5,2023-03-17 17:00:00,2023,3,17,4,11,17,0.00,0.00,0.000000,37.90,21.20,29.540870,57.333333,66.666667,48.086957,31.05,30.25,30.594783,0.000000,0.00000,31.3,0.0,60.0,0.00,9.333333,-16.70,-0.80,False,False,1.945325,False


In [3]:
def forecast_next_24_hours_output_flow_rate(year, month, day, hour, save_df=True):
    water_consumption_silver = pd.read_parquet("../data/silver/water_consumption_silver.parquet")
    original_input_df = pd.read_parquet("../data/silver/training_dataset.parquet")
    timestamp = pd.Timestamp(year=year, month=month, day=day, hour=hour)
    input_df = original_input_df[original_input_df.timestamp == timestamp]
    
    X = input_df[all_features]
    X_no_weather =  input_df[all_non_weather_features] 
    
    # no weather predictions
    predictions = []
    for i in range(1, 25):
        new_prediction = {}
        next_timestamp = timestamp + pd.Timedelta(hours=i)
        model = pickle.load(open(f"../models/no_weather/xgb_{i}h.pkl", "rb"))
        change_from_last_hour_output_flow_rate_mean = model.predict(X_no_weather)[0]
        new_prediction["timestamp"] = next_timestamp
        new_prediction["change_from_last_hour_output_flow_rate_mean"] = change_from_last_hour_output_flow_rate_mean
        predictions.append(new_prediction)
        
    # weather predictions
    weather_predictions = []
    for i in range(1, 25):
        new_prediction = {}
        next_timestamp = timestamp + pd.Timedelta(hours=i)
        model = pickle.load(open(f"../models/weather/xgb_weather_{i}h.pkl", "rb"))
        change_from_last_hour_output_flow_rate_mean = model.predict(X)[0]
        new_prediction["timestamp"] = next_timestamp
        new_prediction["change_from_last_hour_output_flow_rate_mean"] = change_from_last_hour_output_flow_rate_mean
        weather_predictions.append(new_prediction)
        
    predictions = pd.DataFrame(predictions)
    weather_predictions = pd.DataFrame(weather_predictions)
    merged_df = pd.merge(predictions, weather_predictions, on='timestamp', suffixes=('_no_weather', '_weather'))
    
    last_timestamp = merged_df.timestamp.iloc[0]
    first_timestamp = last_timestamp - pd.Timedelta(hours=72)
    timestamps = pd.date_range(start=first_timestamp, end=last_timestamp-pd.Timedelta(hours=1), freq='h')
    water_consumption_silver = water_consumption_silver[water_consumption_silver.timestamp.isin(timestamps)]
    water_consumption_silver = water_consumption_silver[["timestamp", "output_flow_rate_mean"]].rename(columns={"output_flow_rate_mean": "output_flow_rate_mean_no_weather"})
    water_consumption_silver['output_flow_rate_mean_weather'] = water_consumption_silver['output_flow_rate_mean_no_weather']
    water_consumption_silver['forecasted'] = False
    
    for _, row in merged_df.iterrows():
        timestamp = row["timestamp"]
        previous_timestamp = timestamp - pd.Timedelta(hours=1)
        last_output_flow_rate_mean = water_consumption_silver.loc[water_consumption_silver.timestamp == previous_timestamp, 'output_flow_rate_mean_no_weather'].values[0]
        this_hour_output_flow_rate_mean_no_weather = last_output_flow_rate_mean + row["change_from_last_hour_output_flow_rate_mean_no_weather"]
        this_hour_output_flow_rate_mean_weather = last_output_flow_rate_mean + row["change_from_last_hour_output_flow_rate_mean_weather"]
        new_row = {'timestamp': timestamp, 'output_flow_rate_mean_no_weather': this_hour_output_flow_rate_mean_no_weather, 'output_flow_rate_mean_weather': this_hour_output_flow_rate_mean_weather, 'forecasted': True}
        water_consumption_silver = pd.concat([water_consumption_silver, pd.DataFrame(new_row, index=[0])], ignore_index=True)
    
    if not save_df:
        return water_consumption_silver
    
    water_consumption_silver.to_parquet("../data/gold/questions_4_and_7_answers.parquet")
    
def simulate_emptying_reservoir(year, month, day, hour, save_df=True):
    timestamp = pd.Timestamp(year=year, month=month, day=day, hour=hour)
    original_df = pd.read_parquet("../data/silver/water_consumption_silver.parquet")
    original_df = original_df[['timestamp', 'output_flow_rate_mean', 'reservoir_level_percentage_last']]
    original_df = original_df[original_df.timestamp <= timestamp].tail(72)    
    original_df = original_df.rename(columns={'output_flow_rate_mean': 'output_flow_rate_mean_no_weather', 'reservoir_level_percentage_last': 'reservoir_level_percentage_last_no_weather'})
    original_df['output_flow_rate_mean_weather'] = original_df['output_flow_rate_mean_no_weather']
    original_df['reservoir_level_percentage_last_weather'] = original_df['reservoir_level_percentage_last_no_weather']
    original_df['forecasted'] = False
    time_elapsed = 0
    
    while True:
        forecasted_df = forecast_next_24_hours_output_flow_rate(year, month, day, hour, save_df=False)
        timestamp = pd.Timestamp(year=year, month=month, day=day, hour=hour)
        
        for _ in range(24):
            next_timestamp = timestamp + pd.Timedelta(hours=1)
            next_hour_output_flow_rate_mean_no_weather = forecasted_df[forecasted_df.timestamp==next_timestamp].output_flow_rate_mean_no_weather.values[0]
            next_hour_output_flow_rate_mean_weather = forecasted_df[forecasted_df.timestamp==next_timestamp].output_flow_rate_mean_weather.values[0]
            liters_out_next_hour_weather = next_hour_output_flow_rate_mean_weather * 3600
            liters_out_next_hour_no_weather = next_hour_output_flow_rate_mean_no_weather * 3600
            last_reservoir_level_percentage_no_weather = original_df[original_df.timestamp == timestamp].reservoir_level_percentage_last_no_weather.values[0]
            last_reservoir_level_percentage_weather = original_df[original_df.timestamp == timestamp].reservoir_level_percentage_last_weather.values[0]
            next_reservoir_level_percentage_no_weather = last_reservoir_level_percentage_no_weather-((liters_out_next_hour_no_weather/1_000_000)*100)
            next_reservoir_level_percentage_weather = last_reservoir_level_percentage_weather-((liters_out_next_hour_weather/1_000_000)*100)
            new_row = {
                'timestamp': next_timestamp, 
                'output_flow_rate_mean_no_weather': next_hour_output_flow_rate_mean_no_weather, 
                'output_flow_rate_mean_weather': next_hour_output_flow_rate_mean_weather, 
                'reservoir_level_percentage_last_no_weather': next_reservoir_level_percentage_no_weather,
                'reservoir_level_percentage_last_weather': next_reservoir_level_percentage_weather,
                'forecasted': True
            }
            new_row = pd.DataFrame([new_row])
            original_df = pd.concat([original_df, new_row], ignore_index=True)
            timestamp = next_timestamp
            reservoir_level_percentage_no_weather = original_df.reservoir_level_percentage_last_no_weather.tail(1).values[0]
            reservoir_level_percentage_weather = original_df.reservoir_level_percentage_last_weather.tail(1).values[0]
            if reservoir_level_percentage_no_weather > 0 or reservoir_level_percentage_weather > 0:
                time_elapsed += 1
            
        if reservoir_level_percentage_no_weather < 0 or reservoir_level_percentage_weather < 0:
            break 
        
        year = original_df.iloc[-1].timestamp.year
        month = original_df.iloc[-1].timestamp.month
        day = original_df.iloc[-1].timestamp.day
        hour = original_df.iloc[-1].timestamp.hour
        
    first_empty_reservoir_index = original_df[original_df.reservoir_level_percentage_last_no_weather < 0].index[0]
    original_df = original_df.iloc[:first_empty_reservoir_index+1].tail(24).reset_index(drop=True)
    
    if not save_df:
        return original_df, time_elapsed
    
    original_df.to_parquet("../data/gold/question_5_answer.parquet")

In [4]:
# Select the specified columns from the dataframe
selected_columns = [
    'timestamp', 
    'hour', 
    'day_of_week', 
    'week_of_year', 
    'input_flow_rate_first', 
    'input_flow_rate_last', 
    'input_flow_rate_mean', 
    'reservoir_level_percentage_first', 
    'reservoir_level_percentage_last', 
    'reservoir_level_percentage_mean', 
    'output_flow_rate_first', 
    'output_flow_rate_last', 
    'output_flow_rate_mean', 
    'pressure_first', 
    'pressure_last', 
    'pressure_mean', 
    'pump_1_duration_sum', 
    'pump_2_duration_sum', 
    'input_flow_rate_diff', 
    'output_flow_rate_diff', 
    'reservoir_level_change', 
    'pressure_change', 
    'pump_1_active', 
    'pump_2_active'
]

# Create a new dataframe with the selected columns
df_selected = df[selected_columns]

In [5]:
def add_time_to_depletion(df):
    time_to_depletion = []

    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        year = row['timestamp'].year
        month = row['timestamp'].month
        day = row['timestamp'].day
        hour = row['timestamp'].hour

        try:
            _, time_elapsed = simulate_emptying_reservoir(year, month, day, hour, save_df=False)
        except Exception as e:
            time_elapsed = 5
        
        time_to_depletion.append(time_elapsed)
    
    df = df.copy()
    df['time_to_depletion'] = time_to_depletion
    return df

In [6]:
def add_forecasts_to_state(df):
    # Initialize lists to hold the forecast values
    output_flow_rate_forecasts = []
    
    for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing rows"):
        year = row['timestamp'].year
        month = row['timestamp'].month
        day = row['timestamp'].day
        hour = row['timestamp'].hour

        try:
            forecasted_df = forecast_next_24_hours_output_flow_rate(year, month, day, hour, save_df=False)
            forecasted_df = forecasted_df[forecasted_df.forecasted == True]
            output_flow_rate_forecast = forecasted_df['output_flow_rate_mean_no_weather'].values.tolist() # will return 24 values
        except Exception as e:
            output_flow_rate_forecast = [0.0] * 24
            print(f'exception at index {index}')
        output_flow_rate_forecasts.append(output_flow_rate_forecast)
    
    # Add the forecast values to the dataframe
    for i in range(24):
        df[f'forecast_output_flow_rate_{i+1}h'] = [f[i] for f in output_flow_rate_forecasts]

    return df

In [7]:
df_selected = add_forecasts_to_state(df_selected)
df_selected.to_parquet("../data/silver/DQN_states.parquet")

Processing rows:   0%|          | 5/8636 [00:00<12:29, 11.51it/s]  

exception at index 1
exception at index 2
exception at index 3
exception at index 4
exception at index 5
exception at index 6
exception at index 7
exception at index 8


Processing rows:   0%|          | 13/8636 [00:00<06:09, 23.37it/s]

exception at index 9
exception at index 10
exception at index 11
exception at index 12
exception at index 13
exception at index 14
exception at index 15
exception at index 16


Processing rows:   0%|          | 21/8636 [00:01<05:09, 27.82it/s]

...

Processing rows: 100%|█████████▉| 8635/8636 [1:05:33<00:00, 22.99it/s]

exception at index 8628
exception at index 8629
exception at index 8630
exception at index 8631
exception at index 8632
exception at index 8633
exception at index 8634
exception at index 8635


Processing rows: 100%|██████████| 8636/8636 [1:05:33<00:00,  2.20it/s]

exception at index 8636



/var/folders/qv/9ctjx9x12z57ymq41c61h7nr0000gn/T/ipykernel_22277/251084835.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'forecast_output_flow_rate_{i+1}h'] = [f[i] for f in output_flow_rate_forecasts]
/var/folders/qv/9ctjx9x12z57ymq41c61h7nr0000gn/T/ipykernel_22277/251084835.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'forecast_output_flow_rate_{i+1}h'] = [f[i] for f in output_flow_rate_forecasts]
/var/folders/qv/9ctjx9x12z57ymq41c61h7nr0000gn/T/ipykernel_22277/251084835.py:22: Set

In [8]:
# Apply the function to add the time_to_depletion to the dataframe
df_selected = add_time_to_depletion(df_selected)
df_selected.to_parquet("../data/silver/DQN_states.parquet")

Processing rows: 100%|██████████| 8636/8636 [1:12:20<00:00,  1.99it/s]


In [9]:
df_selected.head()

,timestamp,hour,day_of_week,week_of_year,input_flow_rate_first,input_flow_rate_last,input_flow_rate_mean,reservoir_level_percentage_first,reservoir_level_percentage_last,reservoir_level_percentage_mean,output_flow_rate_first,output_flow_rate_last,output_flow_rate_mean,pressure_first,pressure_last,pressure_mean,pump_1_duration_sum,pump_2_duration_sum,input_flow_rate_diff,output_flow_rate_diff,reservoir_level_change,pressure_change,pump_1_active,pump_2_active,forecast_output_flow_rate_1h,forecast_output_flow_rate_2h,forecast_output_flow_rate_3h,forecast_output_flow_rate_4h,forecast_output_flow_rate_5h,forecast_output_flow_rate_6h,forecast_output_flow_rate_7h,forecast_output_flow_rate_8h,forecast_output_flow_rate_9h,forecast_output_flow_rate_10h,forecast_output_flow_rate_11h,forecast_output_flow_rate_12h,forecast_output_flow_rate_13h,forecast_output_flow_rate_14h,forecast_output_flow_rate_15h,forecast_output_flow_rate_16h,forecast_output_flow_rate_17h,forecast_output_flow_rate_18h,forecast_output_flow_rate_19h,forecast_output_flow_rate_20h,forecast_output_flow_rate_21h,forecast_output_flow_rate_22h,forecast_output_flow_rate_23h,forecast_output_flow_rate_24h,time_to_depletion
1,2023-03-17 13:00:00,13,4,11,65.64,65.24,65.587143,39.00,44.40,41.617143,45.640000,49.240000,50.704846,38.13,38.27,38.181429,0.000000,3600.00000,-0.40,3.600000,5.40,0.14,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
2,2023-03-17 14:00:00,14,4,11,66.46,65.64,65.654000,44.40,49.86,46.736500,66.460000,14.858750,50.261628,38.40,38.63,38.459000,0.000000,3600.00000,-0.82,-51.601250,5.46,0.23,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
3,2023-03-17 15:00:00,15,4,11,65.64,33.74,49.395000,50.56,53.80,52.405000,42.306667,93.740000,73.561667,38.70,38.40,38.515000,145.788337,3600.00000,-31.90,51.433333,3.24,-0.30,True,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
4,2023-03-17 16:00:00,16,4,11,37.92,0.00,3.792000,53.80,38.76,45.368000,37.920000,56.000000,46.141631,38.50,31.12,32.054500,0.000000,667.39255,-37.92,18.080000,-15.04,-7.38,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
5,2023-03-17 17:00:00,17,4,11,0.00,0.00,0.000000,37.90,21.20,29.540870,57.333333,66.666667,48.086957,31.05,30.25,30.594783,0.000000,0.00000,0.00,9.333333,-16.70,-0.80,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5
